In [2]:
# This file produces lower bounds on the multiplicity of the first Laplacian eigenvalue of triangle surfaces
# based on the representation theory of their isometry group or the subgroup of orientation-preserving isometries

##########################################################################################################
# -----------------------------------------------------------------
#  CCdV
# -----------------------------------------------------------------
#
# INPUT:
# - g : a genus
#
# OUTPUT:
# - The multiplicity for the examples constructed by Colbois and Colin de Verdière
##########################################################################################################
def CCdV(g):
    if g==2:
        #Their construction does not work in genus 2 so we return 1
        return 1
    else:
        return floor((1+sqrt(8*g+1))/2)

##########################################################################################################
# -----------------------------------------------------------------
#  min_dim
# -----------------------------------------------------------------
#
# INPUT:
# - G : a group
#
# THE PROGRAM:
# - Computes the character table of G.
# - Multiplies the dimension of each irreducible representation with an element of non-real trace by 2
# - Returns the smallest real dimension > 1 encountered.
# 
# OUTPUT:
# - the smallest real dimension > 1 found    
##########################################################################################################
def min_dim(G):    
    #Convert G to a GAP object 
    G = G.gap()
    
    #Compute its character table
    C = G.CharacterTable()

    #Retrieve the character degrees
    #This is a list of pairs where the first (index 0) entry is the degree
    #and the second (index 1) is the number of times this degree appears 
    D = C.CharacterDegrees()
    
    #We then form the list of degrees repeated with multiplicity and we multiply them all by 2
    degrees = []
    for pair in D:
        deg = pair[0]
        number = pair[1]
        for k in range(number):
            degrees.append(2*deg)
    
    #We find which characters are real-valued and divide their degree by 2
    #because the dimension over R is equal to the dimension over C in these cases
    R = C.RealClasses()
    for j in R:
        degrees[j-1] = degrees[j-1]/2
    
    #Sort the list of real degrees in increasing order
    degrees.sort()
    
    #Find the smallest degree > 1
    k=0
    while degrees[k]==1:
        k = k+1
        
    #Clear out the memory in GAP
    
    
    #Return the smallest degree > 1
    return degrees[k]

    
##########################################################################################################
# -----------------------------------------------------------------
#  max_min_dim
# -----------------------------------------------------------------
#
# INPUT:
# - orders : the triple (p,q,r) of orders of the generators x, y, z for the triangle group
# - relations : a string representing the list of relations for the triangle group quotient
# - reflexible : a boolean saying whether or not the group action can be extended with reflections
#                in the sides of a fundamental triangle
#
# THE PROGRAM:
# - Creates the group from the generators and relations
# - Checks that the hypothesis of Lemma 9.2 is satisfied for G = Isom+ or the hypothesis of [FBP21, Prop 4.4] for H = Isom.
# - Calls the function min_dim for each group that satisfies the requirements
# - Returns the largest of the two values obtained
# 
# OUTPUT:
# - a tuple (max_min,orientation) where: - max_min, the smallest real dimension >1 of an irreducible representation
#                                          for the triangle group quotient or its kaleidoscopic extension
#                                          (when it exists), whichever is largest
#                                        - orientation, a string equal to 'rotation' if the above is achieved for the triangle group
#                                          and equal to 'reflection' if it is achieved for the kaleidoscopic extension     
##########################################################################################################
def max_min_dim(orders,relations,reflexible):
    
    # Define the automorphism group as a quotient of F_3 by the relations
    F.<x,y,z> = FreeGroup()
    #Convert the string of relations to a sage object 
    rel = sage_eval(relations,cmds='F.<x,y,z>=FreeGroup()')
    G = F/rel
    
    # Compute the cardinality of the group
    n = G.cardinality()
    (p,q,r)=orders
    
    #This is in case the area condition fails
    min_dim_oriented = 0
    
    # If the area condition is satisfied, 
    # go through the character table and keep track of the smallest dimension greater than 1 encountered
    if n >= 3*r:        
        min_dim_oriented = min_dim(G)
    
    #This is in case the action is not reflexible or the area test fails
    min_dim_nonoriented = 0
    
    if reflexible == True:

        # If reflexible is True, we construct the extended isometry group
        # a, b, c are reflections in the sides of a fundamental triangle such that ab=x, bc=y, ca=z
        FF.<a,b,c> = FreeGroup()
        
        relations2 = relations.replace('x','(a*b)')
        relations2 = relations2.replace('y','(b*c)')
        relations2 = relations2.replace('z','(c*a)')
        
        #Convert the string to a sage object 
        rel2 = sage_eval(relations2,cmds='FF.<a,b,c>=FreeGroup()')
        H = FF/(rel2+[a^2,b^2,c^2])
        
        # Compute the cardinality of the group
        m = H.cardinality()

        # If the area condition is satisfied, 
        # go through the character table and keep track of the smallest dimension greater than 1 encountered
        if m >= 6*r:            
            min_dim_nonoriented = min_dim(H)
    
    #Record which one of the two groups gives the largest answer
    max_min = max(min_dim_oriented,min_dim_nonoriented)
                      
    if max_min == min_dim_oriented:
        orientation = 'rotation'
    else:
        orientation = 'reflection'
    
    return (max_min,orientation)

##########################################################################################################
# -----------------------------------------------------------------
#  find_best
# -----------------------------------------------------------------
#
# INPUT:
# - g : a genus
# - L : the list of triangle group quotients in genus g. Every entry of L is assumed to be 
#       a triple (orders,relations,reflexible) where:
#                                                     - orders = (p,q,r) are the orders of the generators
#                                                     - relations is a string representing the relations that define the group 
#                                                       as a quotient of the free group <x,y,z>
#                                                     - reflexible is a boolean saying whether or not the action can be extended 
#  
# THE PROGRAM:
# - Produces the label associated to each entry in L in Conder's list
# - Calls max_min_dim on each entry in L
# - Records the largest answer found and its attributes, prints them out,
#   and compares the answer with the examples of Colbois and Colin de Verdière 
#
# OUTPUT:
# - best : the largest lower bound on multiplicity obtained among the groups in L and their extensions
##########################################################################################################                   
def find_best(g,L):
    #Keep track of the best lower bound so far
    best = 0
    beat_orders = None
    best_orientation = None
    best_label = None 
                      
    #Go through all entries in the list
    for k in range(len(L)):
        (orders,relations,reflexible) = L[k]
        #Create the label associated to the entry
        label = 'T{0}.{1}'.format(g,k+1)
        result = max_min_dim(orders,relations,reflexible)
                      
        #If we beat the previous record, we save the characteristics of the new record holder
        if result[0]>best:
            best = result[0]
            best_orders = orders
            best_orientation = result[1]
            best_label = label
     
    return (best,best_orders,best_orientation,best_label)

In [3]:
# T = triangle_quotients.sobj contains the list of all triangle quotients in genus 2 to 20 extracted from Conder's list
# The g-2 entry of T is the list of all groups in genus g
# Each enty in T[g-2] is a tuple (orders,relations,reflexible) where
# orders = (p,q,r) are the orders of the generators x, y, z for the triangle group
# relations is a string representing the relations defining the group
# reflexible is a boolean, set to True if the group is reflexible[m] in Conder's notation, False otherwise

T = load('triangle_quotients')

In [4]:
# In our experience, running this cell uses a lot of memory. In particular, on some of the computers we
# ran this on, we needed to split the range below into multiple subranges in order to run all genera. 
for g in range(2,21):
    (best,best_orders,best_orientation,best_label)=find_best(g,T[g-2])
    print('The best lower bound on multiplicity found in genus',g,'is',best,'while Colbois and Colin de Verdière obtain',CCdV(g))
    if best > 0:
        print('It is achieved by the triangle',best_orientation,'group of type',best_orders,'with label',best_label,'in Conder\'s list.\n')

The best lower bound on multiplicity found in genus 2 is 2 while Colbois and Colin de Verdière obtain 1
It is achieved by the triangle rotation group of type (2, 3, 8) with label T2.1 in Conder's list.

The best lower bound on multiplicity found in genus 3 is 6 while Colbois and Colin de Verdière obtain 3
It is achieved by the triangle reflection group of type (2, 3, 7) with label T3.1 in Conder's list.

The best lower bound on multiplicity found in genus 4 is 4 while Colbois and Colin de Verdière obtain 3
It is achieved by the triangle rotation group of type (2, 4, 5) with label T4.2 in Conder's list.

The best lower bound on multiplicity found in genus 5 is 3 while Colbois and Colin de Verdière obtain 3
It is achieved by the triangle rotation group of type (2, 3, 10) with label T5.2 in Conder's list.

The best lower bound on multiplicity found in genus 6 is 4 while Colbois and Colin de Verdière obtain 4
It is achieved by the triangle rotation group of type (2, 4, 6) with label T6.2 i